portrai-batch: 用户画像更新
1. 输入：
    1. action.csv
    2. raw_embed_user_mapping.pickle
    3. raw_embed_item_mapping.pickle
    4. portrait.pickle （if new user added, it will update）
    5. news_id_news_property_dict.pickle
2. 输出
    1. portrait.pickle

In [1]:
from __future__ import print_function
import os
import sys
import math
import pickle
import boto3
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import argparse
import logging
import re

# tqdm.pandas()
# pandarallel.initialize(progress_bar=True)
# bucket = os.environ.get("BUCKET_NAME", " ")
# raw_data_folder = os.environ.get("RAW_DATA", " ")

s3client = boto3.client('s3')

In [2]:
########################################
# 从s3同步数据
########################################
s3client = boto3.client('s3')


def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))


def write_to_s3(filename, bucket, key):
    print("upload s3://{}/{}".format(bucket, key))
    with open(filename, 'rb') as f:  # Read in binary mode
        # return s3client.upload_fileobj(f, bucket, key)
        return s3client.put_object(
            ACL='bucket-owner-full-control',
            Bucket=bucket,
            Key=key,
            Body=f
        )

def write_str_to_s3(content, bucket, key):
    print("write s3://{}/{}, content={}".format(bucket, key, content))
    s3client.put_object(Body=str(content).encode("utf8"), Bucket=bucket, Key=key, ACL='bucket-owner-full-control')

default_bucket = 'aws-gcr-rs-sol-demo-ap-southeast-1-522244679887'
default_prefix = 'sample-data'
parser = argparse.ArgumentParser()
parser.add_argument('--bucket', type=str)
parser.add_argument('--prefix', type=str)
args, _ = parser.parse_known_args()
bucket = args.bucket
prefix = args.prefix

print("bucket={}".format(bucket))
print("prefix='{}'".format(prefix))

print("bucket={}".format(bucket))
print("prefix='{}'".format(prefix))

s3client = boto3.client('s3')
local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)
# 行为数据加载
file_name_list = ['action.csv']
s3_folder = '{}/system/action-data'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
# # 用户画像数据加载
# file_name_list = ['portrait.pickle']
# s3_folder = '{}/feature/recommend-list/portrait'.format(prefix)
# sync_s3(file_name_list, s3_folder, local_folder)
# 倒排列表的pickle文件
file_name_list = ['news_id_news_property_dict.pickle']
s3_folder = '{}/feature/content/inverted-list/'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)

bucket=aws-gcr-rs-sol-demo-ap-southeast-1-522244679887
prefix='sample-data'
bucket=aws-gcr-rs-sol-demo-ap-southeast-1-522244679887
prefix='sample-data'
file preparation: download src key sample-data/system/action-data/action.csv to dst key info/action.csv
file preparation: download src key sample-data/feature/content/inverted-list/news_id_news_property_dict.pickle to dst key info/news_id_news_property_dict.pickle


In [3]:
# 加载用户数据
file_to_load = open("info/news_id_news_property_dict.pickle", "rb")
dict_id_content = pickle.load(file_to_load)

user_click_records = {}
data_mk = pd.read_csv('info/action.csv', sep='_!_',names=['user_id','news_id','timestamp','action_type','action'])
for reviewerID, hist in tqdm(data_mk[(data_mk['action'] == 1) & (data_mk['action_type'] == 1)].groupby('user_id')):
    pos_list = hist['news_id'].tolist()
    user_click_records[reviewerID] = pos_list

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
100%|██████████| 997/997 [00:00<00:00, 8127.28it/s]


In [4]:
########################################
# 初始化用户画像
# 数据结构:
# 'language':{'xx':{'mark':,'score':},...'recent':['xx',score]}}
# 'embedding':{'review':xxx,'photo':xxx,'ub':xxx}
########################################
def initial_user_portrait(user_list):
    user_data_frame = {}
    popularity_method_list = ['keywords', 'type']
    for user in user_list:
        current_user_portrait = {}
        for mt in popularity_method_list:
            current_user_portrait[mt] = {}
            current_user_portrait[mt]['recent'] = []
            current_user_portrait[mt]['recent'].append([])
            current_user_portrait[mt]['recent'].append(0.0)
        user_data_frame[str(user)] = current_user_portrait
    #     col_name = ['user_id']+popularity_method_list
    #     portrait_pddf = pd.DataFrame.from_dict(user_data_frame, orient='index',
    #                            columns=col_name)
    #     portrait_pddf = portrait_pddf.reset_index(drop=True)
    #     portrait_pddf.to_csv('info/portrait.csv')
    # save pickle file
    file_name = 'info/portrait.pickle'
    output_file = open(file_name, 'wb')
    pickle.dump(user_data_frame, output_file)
    output_file.close()
    # # backup original file
    # file_name = 'info/user_portrait_{}_raw.pickle'.format(user_name)
    # output_file = open(file_name, 'wb')
    # pickle.dump(current_user_portrait, output_file)
    # output_file.close()

initial_user_portrait(list(user_click_records.keys()))

In [5]:
def update_portrait_under_a_property(mt_content, mt_up, ratio):
    # decay logic
    for k, v in mt_up.items():
        if k != 'recent':
            if mt_up[k]['mark'] != '1':
                mt_up[k]['score'] = mt_up[k]['score'] * ratio
            else:
                mt_up[k]['mark'] = '0'

    # update logic
    for ct in mt_content:
        if ct != None:
            if ct not in mt_up:
                mt_up[ct] = {}
                mt_up[ct]['mark'] = '1'
                mt_up[ct]['score'] = 1.0
            else:
                mt_up[ct]['mark'] = '1'
                mt_up[ct]['score'] = mt_up[ct]['score'] + 1.0

    # find large score
    for k, v in mt_up.items():
        # update large score and type
        if k != 'recent':
            if mt_up[k]['score'] >= mt_up['recent'][1] and k not in mt_up['recent'][0]:
                mt_up['recent'][0].append(k)
                mt_up['recent'][1] = mt_up[k]['score']


########################################
# 用户画像更新逻辑
# 数据结构:
# 'language':{'xx':{'mark':,'score':},...{'recent':['xx',score]}}
# 'embedding':{'review':xxx,'photo':xxx,'ub':xxx}
########################################
def update_user_portrait_with_one_click(current_user_portrait, current_read_item):
    #     # load user portrait for testing
    #     file_to_load = open("info/user_portrait_{}.pickle".format(user_name), "rb")
    #     current_user_portrait = pickle.load(file_to_load)
    #     print("load user portrait of the content is {}".format(current_user_portrait))

    # 用户兴趣衰减系数
    decay_ratio = 0.8

    popularity_method_list = ['keywords','type']

    for mt in popularity_method_list:
        update_portrait_under_a_property(
            dict_id_content[current_read_item][mt], current_user_portrait[mt], decay_ratio)


#     print("updated user portrait {}".format(current_user_portrait))
#     # save pickle files
#     file_name = 'info/user_portrait_{}.pickle'.format(user_name)
#     output_file = open(file_name, 'wb')
#     pickle.dump(current_user_portrait, output_file)
#     output_file.close()


def update_user_embedding(user_id, input_item_list):
    #     file_to_load = open("info/user_portrait_{}.pickle".format(user_id), "rb")
    #     current_user_portrait = pickle.load(file_to_load)
    #     print("load user portrait of {}, the content is {}".format(
    #         user_name, current_user_portrait))

    # 映射用户的embedding
    # 构建适合模型的输入
    watch_list_len = 50
    map_input_item_list = np.array([[0] * watch_list_len])
    watch_len = len(input_item_list)
    map_user_id = dict_user_mapping[str(user_id)]
    for cnt, item in enumerate(input_item_list):
        if cnt < 50:
            map_input_item_list[0][cnt] = dict_item_mapping[str(item)]
    model_input = {}
    model_input['user_id'] = np.array([int(map_user_id)])
    model_input['hist_movie_id'] = map_input_item_list
    model_input['hist_len'] = np.array([watch_len])

    # 更新用户的embeddings
    #     print("model input {}".format(model_input))
    updated_user_embs = user_embedding_model.predict(
        model_input, batch_size=2 ** 12)

    #     current_user_portrait['ub_embed'] = updated_user_embs

    #     print("update user embeddings {}".format(updated_user_embs))

    return updated_user_embs


file_to_load = open("info/portrait.pickle", "rb")
dict_user_portrait = pickle.load(file_to_load)
print("update user portrait for batch users")
for user_id, input_item_list in user_click_records.items():
    print("user id {} item list {}".format(user_id, input_item_list))
    # update click history
    dict_user_portrait[str(user_id)]['click_sets'] = input_item_list
    for ci in input_item_list:
        update_user_portrait_with_one_click(dict_user_portrait[str(user_id)], str(ci))

# 存储和更新用户画像
file_name = 'info/portrait.pickle'
output_file = open(file_name, 'wb')
pickle.dump(dict_user_portrait, output_file)
output_file.close()
write_to_s3(file_name,
            bucket,
            "{}/feature/recommend-list/portrait/{}".format(prefix, file_name.split('/')[-1]))

update user portrait for batch users
user id 52a23654-9dc3-11eb-a364-acde48001122 item list [6475484594673025293, 6552473089853096452, 6533143503382249998]
user id 52a238fc-9dc3-11eb-a364-acde48001122 item list [6553127862163997192, 6552330831065514503, 6552410777486098952, 6552469334151660045, 6552331253163491848]
user id 52a239c4-9dc3-11eb-a364-acde48001122 item list [6552411965111665156, 6503824853412348174, 6552388108204114435, 6552323120307372548, 6552257299056951815, 6552207083696030216]
user id 52a23a5a-9dc3-11eb-a364-acde48001122 item list [6470669782923346189, 6553404738497413636, 6552276082224005636, 6552416603969946120, 6552351377643799044, 6472907515020443918, 6552478601705423374, 6551709208545329678]
user id 52a23adc-9dc3-11eb-a364-acde48001122 item list [6552367183177974275, 6552295529848504836, 6552487380748599811, 6552291867231257096, 6553021200635789832, 6552261008373580301, 6552269216773308935, 6552378967637099011]
user id 52a23b54-9dc3-11eb-a364-acde48001122 item lis

user id 52a3abec-9dc3-11eb-a364-acde48001122 item list [6552440903414317576, 6552334864228549133, 6552466727995703815, 6552353298312069640, 6552853903359083015]
user id 52a3ac50-9dc3-11eb-a364-acde48001122 item list [6533008673860485639, 6551657353450619400, 6552458394618298883, 6543758118407373319, 6552459332552753672, 6546844349055894023, 6521968288589677060, 6552434358081290766, 6552146340690788877]
user id 52a3acbe-9dc3-11eb-a364-acde48001122 item list [6552280932433265156, 6552460748696584717, 6552334949041570317, 6552391240820720132, 6552446105915752973, 6552422704941629966, 6552452132908302856]
user id 52a3ad2c-9dc3-11eb-a364-acde48001122 item list [6552449933050380804, 6552490922678944259, 6553126573522813454, 6552999862319710733, 6552232931102818824]
user id 52a3ad90-9dc3-11eb-a364-acde48001122 item list [6553132214173827592, 6521606799425012232, 6552260603971371533]
user id 52a3adfe-9dc3-11eb-a364-acde48001122 item list [6552384992020791815, 6552470938275807758, 6552463419860

{'ResponseMetadata': {'RequestId': 'FS7J2F5E4S7JF019',
  'HostId': 'iTlj4nj9Rw4zBo1UB3N/SgjdvcVUZPZWHGK24R1ZhPswEqR94jqV3uX6jRc2KanNJV5NOD2Cm3Q=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'iTlj4nj9Rw4zBo1UB3N/SgjdvcVUZPZWHGK24R1ZhPswEqR94jqV3uX6jRc2KanNJV5NOD2Cm3Q=',
   'x-amz-request-id': 'FS7J2F5E4S7JF019',
   'date': 'Mon, 19 Apr 2021 03:21:10 GMT',
   'etag': '"b089ff2aa0a8f25626e0790dbf85f675"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"b089ff2aa0a8f25626e0790dbf85f675"'}

In [6]:
n = 0
for k, v in rd['news_id_news_property_dict'].items():
    print("k {} v {}".format(k,v))
    if n > 10:
        break
    n = n + 1

NameError: name 'rd' is not defined